## Домашняя работа 11 - Функции и работа с данными
### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [127]:
import pandas as pd

In [128]:
ratings = pd.read_csv('.\\HW-11 data\\ml-latest-small\\ratings.csv')

In [129]:
def rate_class(rating):
    return {
           rating <= 2: 'низкий рейтинг', 
        2 < rating <= 4:    'средний рейтинг',
        4.5 <= rating <= 5:  'высокий рейтинг'
    }[True]

In [130]:
ratings['class'] = ratings['rating'].apply(rate_class) 
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [131]:
keywords = pd.read_csv('.\\HW-11 data\\ml-latest-small\\keywords.csv')

In [132]:
import re
def get_region(string):
    str_list = re.findall(r'\w+', string)
    for word in str_list:
        if word.lower() in ['москва', 'тула', 'ярославль']:
            return 'Центр'
        elif word.lower() in ['петербург', 'псков', 'мурманск']:
            return 'Северо-Запад'
        elif word.lower() in ['владивосток', 'сахалин', 'хабаровск']:
            return 'Дальний Восток'
    return 'undefined'

In [133]:
keywords['region'] = keywords['keyword'].apply(get_region)
check = keywords[keywords['region'] != 'undefined']
check.head(10)

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
1259,москва,153050,Центр
1346,санкт петербург,144232,Северо-Запад
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр
1579,авито тула,126168,Центр


### Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

- В переменную years запишите список из всех годов с 1950 по 2010.

- Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

- Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

- Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга


In [123]:
movies = pd.read_csv('.\\HW-10 data\\ml-25m\\movies.csv')
ratings = pd.read_csv('.\\HW-11 data\\ml-latest-small\\ratings.csv')

In [124]:
def production_year(string):
    year =0
    try:
        year = re.search(r'(?!\()\d{4}(?=\))',string).group(0)
    except: pass
    if int(year) in range(1950,2011):
        return int(year)
    else:
        return 1900

In [125]:
data = ratings.merge(movies, on="movieId", how="left")
data['year'] = data['title'].apply(production_year)
data.head(5)


,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [126]:
result = (data[['year','rating']].groupby(['year']).mean()
 .sort_values('rating',ascending=False)
)
result.head(10)

,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.992754
1951,3.983539
1974,3.978625
1962,3.952316
1950,3.915254
1977,3.905506
